In [11]:
'''
流程：1.准备样本 2.训练 3.预测
正样本（820个）：pos包含所要检测的目标对象 负样本（1931个）：neg不包含所要检测的目标对象
正负样本比例：1:2 ~ 1:3之间
正样本应该尽可能的多样：环境多样 干扰多样
一个好的样本远胜于复杂的神经网络，机器学习需要的样本比深度学习少
收集样本：1.向相关公司购买 2.网络爬取数据 3.公司内部多年积累的数据
'''
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1.参数设置
# 正负样本个数
PosNum = 820
NegNum = 1931
# 窗体大小
winSize = (64,128)
# block大小以及步长
blockSize = (16,16)
blockStride = (8,8)
# cell范围
cellSize = (8,8)
# bin数量
nBin = 9
# 2.HOG创建
hog = cv2.HOGDescriptor(winSize,blockSize, blockStride,cellSize,nBin)
# 3.SVM的创建
svm = cv2.ml.SVM_create()
# 4.计算hog(3780)并标注标签
featureNum = int(((128-16)/8+1)*((64-16)/8+1)*4*9) 
featureArray = np.zeros(((PosNum+NegNum),featureNum),np.float32)
labelArray = np.zeros(((PosNum+NegNum),1),np.int32)
# 计算正样本hog特征
for i in range(0,PosNum):
    fileName = 'pos\\'+str(i+1)+'.jpg'
    img = cv2.imread(fileName,1)
    hist = hog.compute(img,(8,8))
    # hog特征装载
    for j in range(0,featureNum):
        featureArray[i,j] = hist[j] 
    # 正样本label为1
    labelArray[i,0] = 1
# 计算负样本hog特征
for i in range(0,NegNum):
    fileName = 'neg\\'+str(i+1)+'.jpg'
    img = cv2.imread(fileName,1)
    hist = hog.compute(img,(8,8))
    # hog特征装载
    for j in range(0,featureNum):
        featureArray[i+PosNum,j] = hist[j] 
    # 负样本label为-1
    labelArray[i+PosNum,0] = -1
# SVM属性设置
svm.setType(cv2.ml.SVM_C_SVC)
# 设置内核为线性分类器
svm.setKernel(cv2.ml.SVM_LINEAR)
svm.setC(0.01)
# 5.训练
result = svm.train(featureArray,cv2.ml.ROW_SAMPLE,labelArray)
# 6.检测（创建myHog进行检测）
alpha = np.zeros((1),np.float32)
# 得到hog描述信息用于判决
rho = svm.getDecisionFunction(0,alpha)
alphaArray = np.zeros((1,1),np.float32)
svmArray = np.zeros((1,featureNum),np.float32)
# 3780维
resultArray = np.zeros((1,featureNum),np.float32)
alphaArray[0,0] = alpha
# 支持向量的个数
resultArray = -1*alphaArray*svmArray
# 3780 + 1 = 3781维 3780来自resultArray 1来自rho
detect = np.zeros((3781),np.float32)
for i in range(0,3780):
    detect[i] = resultArray[0,i]
detect[3780] = rho[0]
myHog = cv2.HOGDescriptor()
myHog.setSVMDetector(detect)
# 加载待检测图片
imgSrc = cv2.imread('Test2.jpg',1)
# 检测出目标的信息 
# 参数1：图片 参数2: 灰度图像 参数3：winStride 参数4：窗体大小 参数5：缩放系数 参数6：目标大小单位为像素
objects = myHog.detectMultiScale(imgSrc,0,(8,8),(32,32),1.05,2)
# 起始位置以及宽高x,y,w,h存放在最后一维
x = int(objects[0][0][0])
y = int(objects[0][0][1])
w = int(objects[0][0][2])
h = int(objects[0][0][3])
# 绘制并展示
cv2.rectangle(imgSrc,(x,y),(x+w,y+h),(255,0,0),2)
cv2.imshow('dst',imgSrc)
cv2.waitKey(0)


-1